In [1]:
import CoolProp.CoolProp as CP
import pandas as pd
import numpy as np

import plotly.graph_objs as go


fluids = ['Water', 'Ammonia', 'R134a']

for fluid in fluids:
    T_min = CP.PropsSI('Tmin', fluid) + 10  # Adding a small offset to Tmin
    T_max = CP.PropsSI('Tcrit', fluid)
    T_vals = np.linspace(T_min, T_max, 100)  # Number of points

    t_s_liquid = {'Temperature (K)': [], 'Entropy Liquid (J/kg.K)': []}
    t_s_vapor = {'Temperature (K)': [], 'Entropy Vapor (J/kg.K)': []}

    for T in T_vals:
        s_liquid = CP.PropsSI('S', 'T', T, 'Q', 0, fluid)  # Saturated liquid
        s_vapor = CP.PropsSI('S', 'T', T, 'Q', 1, fluid)  # Saturated vapor
        t_s_liquid['Temperature (K)'].append(T)
        t_s_liquid['Entropy Liquid (J/kg.K)'].append(s_liquid)
        t_s_vapor['Temperature (K)'].append(T)
        t_s_vapor['Entropy Vapor (J/kg.K)'].append(s_vapor)

    # Save to CSV
    pd.DataFrame(t_s_liquid).to_csv(f'{fluid}_T_s_liquid.csv', index=False)
    pd.DataFrame(t_s_vapor).to_csv(f'{fluid}_T_s_vapor.csv', index=False)


for fluid in fluids:
    df_liquid = pd.read_csv(f'{fluid}_T_s_liquid.csv')
    df_vapor = pd.read_csv(f'{fluid}_T_s_vapor.csv')

    # Creating Plotly plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_liquid['Entropy Liquid (J/kg.K)'], y=df_liquid['Temperature (K)'],
                             mode='lines', name='Saturated Liquid'))
    fig.add_trace(go.Scatter(x=df_vapor['Entropy Vapor (J/kg.K)'], y=df_vapor['Temperature (K)'],
                             mode='lines', name='Saturated Vapor'))

    fig.update_layout(title=f'{fluid} T-s Diagram', 
                      xaxis_title='Entropy (J/kg.K)', yaxis_title='Temperature (K)')

    # Save as HTML
    fig.write_html(f'{fluid}_T_s_plot.html')